In [3]:
import geemap
geemap.update_package()
import geemap.foliumap as geemap
import ee
import geemap
import geemap.colormaps as cm
Map = geemap.Map(center=[40,-100], zoom=4)
Map
import pandas as pd

Unzipping geemap-master.zip ...
Data downloaded to: C:\Users\yoah2447\Downloads\geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


# mapping landcover 2016 and state boundary

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map
#dem = ee.Image('USGS/SRTMGL1_003')
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');
Map.addLayer(landcover, {}, 'landcover')

# Map.addLayer(
#     landcover,
#     {'bands': ['landcover'], 'min': 0, 'max': 95},
#     'landcover',
# )
Map

#county = ee.Image("TIGER/2016/Counties") #.filter(ee.Filter.eq('STATEFP', '23'));
county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', '23'));
#county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', '103')).filter(ee.Filter.eq('STATEFP20', '13'));
#county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));

Map.addLayer(county, {}, 'US States')
Map

## mapping landcover 2019 and state boundary

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = dataset.select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')
Map.add_legend(builtin_legend='NLCD')
Map

# zonal stat 

# census tract level (2010 census boundary)

In [ ]:
DF = pd.read_csv('D:/DATA/NHGIS/nhgis0026_shape (1)/nhgis0026_shape/nhgis0026_shapefile_tl2010_us_tract_2010/US_tract_2010.csv')

DF['GEOID'] =DF['GEOID'].apply(lambda x: '{0:0>11}'.format(x))
DF

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');
# Map.addLayer(landcover, {}, 'landcover')
# i=1
# states = ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));
# Map.addLayer(states, {}, 'US States')
# Map.add_legend(builtin_legend='NLCD')
# Map


for i in range(0,len(DF)):
    county = ee.FeatureCollection("TIGER/2010/Tracts_DP1").filter(ee.Filter.eq('geoid10', DF['GEOID'][i]));
    #county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    #county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './censustract2010boundary/lu_stats'+str(DF['GEOID'][i])+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

# census tract level (2020 census boundary)

In [4]:
DF = pd.read_csv('./countyfips.csv')
DF['STATEFP'] =DF['STATEFP'].apply(lambda x: '{0:0>2}'.format(x))
DF['COUNTYFP'] =DF['COUNTYFP'].apply(lambda x: '{0:0>3}'.format(x))
DF.loc[932]
#DF.loc[DF.GEOID ==78010]
len(DF)

3233

In [ ]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');


for i in range(0,len(DF)):
    county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', fips));
    #county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    #county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './censustract2020boundary/lu_stats'+DF['STATEFP'][i]+ DF['COUNTYFP'][i]+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

# Census block level (2010 census boundary) 

In [ ]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');

for i in range(0,len(DF)):
    #county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', fips));
    #county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './censustract2010boundary/lu_stats'+DF['STATEFP'][i]+ DF['COUNTYFP'][i]+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

# census block level (2020 census boundary)

In [ ]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');


for i in range(0,len(DF)):
    #county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', fips));
    county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    #county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './censustract2010boundary/lu_stats'+DF['STATEFP'][i]+ DF['COUNTYFP'][i]+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass